In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('treebank')
nltk.download('punkt')
from nltk.corpus import treebank_raw
from nltk import NaiveBayesClassifier
from nltk.classify import accuracy as nltk_accuracy
from nltk import word_tokenize, sent_tokenize

In [ ]:
sents = treebank_raw.sents()
tokens = []
boundaries = set()
offset = 0
for sent in sents:
    tokens.extend(sent)
    offset += len(sent)
    boundaries.add(offset - 1)

In [ ]:
def punct_features(tokens, i):
    return {'next-word-capitalized': tokens[i + 1][0].isupper(),
           'prev-word': tokens[i - 1].lower(),
           'punct': tokens[i],
           'prev-word-is-one-char': len(tokens[i - 1]) == 1}

In [ ]:
featuresets = [(punct_features(tokens, i), (i in boundaries))
              for i in range(1, len(tokens) - 1)
              if tokens[i] in '.?!']

In [ ]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = NaiveBayesClassifier.train(train_set)
print(nltk_accuracy(classifier, test_set))

In [ ]:
def segment_sentences(words):
    start = 0
    sents = []
    for i, word in enumerate(words):
        if i == len(words) - 1:
            break
        if word in '.?!' and classifier.classify(punct_features(words, i)) == True and i < len(words):
            sents.append(words[start:i+1])
            start = i + 1
    if start < len(words):
        sents.append(words[start:])
    return [' '.join(sent) for sent in sents]

In [ ]:
test_sent_raw = 'This is the first sentence! This is an abbreviation U.S.A. so it is not the second sentence.'
test_sent_tokens = word_tokenize(test_sent_raw)
print(segment_sentences(test_sent_tokens))

In [ ]:
print(sent_tokenize(test_sent_raw))